In [1]:
from bayes_opt import BayesianOptimization
import numpy as np
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
from matplotlib import gridspec

from trainlib.ConfigFileHandler import ConfigFileHandler

In [2]:
def load_file(path, keys):
    confhandler = ConfigFileHandler()
    confhandler.load_configuration(path)
    
    retval = {}
    
    for section_name in confhandler.get_sections():
        cur_section = confhandler.get_section(section_name)
        
        for key in keys:
            if not key in retval:
                retval[key] = []
                
            retval[key].append(float(cur_section[key]))
            
    return retval

In [45]:
def posterior(bo, x):
    bo.gp.fit(bo.X, bo.Y)
    mu, sigma = bo.gp.predict(x, return_std=True)
    return mu, sigma

def plot_gp(bo, x):
    
    fig = plt.figure()
    axis = plt.subplot(111)
    
    mu, sigma = posterior(bo, x)
    axis.plot(bo.X.flatten(), bo.Y, 's', markersize=3, label=u'observations', color='black')
    axis.plot(x, mu, '--', color='black', label='gaussian process mean')

    axis.fill(np.concatenate([x, x[::-1]]), 
              np.concatenate([mu - 1.9600 * sigma, (mu + 1.9600 * sigma)[::-1]]),
        alpha=.6, fc='lightblue', ec='None', label='95% confidence interval')
    
    axis.set_xlim((min(x), max(x)))
    axis.set_ylim((None, None))
    axis.set_ylabel('utility')
    axis.set_xlabel('ggH prior')
    
    axis.legend(loc = 3)
    
    return fig

In [46]:
init_dict = load_file("/data_CMS/cms/wind/180430_combined_noZZ/number_layers_2.0_number_neurons_64.0/priors_tree/evaluations_ggH.txt", ["target", "ggh_prior"])

attempting to load configuration file from /data_CMS/cms/wind/180430_combined_noZZ/number_layers_2.0_number_neurons_64.0/priors_tree/evaluations_ggH.txt


In [47]:
gp_params = {'kernel': 1.0 * Matern(length_scale = 0.01, length_scale_bounds = (1e-5, 1e5), nu = 1.5),
             'alpha':1.5e-6}

In [48]:
target = lambda x: x

In [49]:
x = np.linspace(1.0, 3.0, 1000).reshape(-1,1)

In [50]:
bo = BayesianOptimization(target, {'ggh_prior': (1.0, 3.0)})

In [51]:
bo.initialize(init_dict)

In [52]:
bo.maximize(init_points=0, n_iter=0, acq='poi', kappa=5, xi = 0.1, **gp_params)

Initialization
-------------------------------------------
 Step |   Time |      Value |   ggh_prior | 
    1 | 00m00s |   -0.01004 |      2.4430 | 
    2 | 00m00s |    0.01100 |      1.5032 | 
    3 | 00m00s |    0.01055 |      1.5032 | 
    4 | 00m00s |    0.00625 |      1.7451 | 
    5 | 00m00s |    0.00253 |      1.0000 | 
    6 | 00m00s |   -0.03193 |      3.0000 | 
    7 | 00m00s |    0.00377 |      2.0929 | 
    8 | 00m00s |   -0.00629 |      2.4223 | 
    9 | 00m00s |    0.00826 |      1.6891 | 
   10 | 00m00s |   -0.02198 |      2.7561 | 
   11 | 00m00s |   -0.03045 |      2.9002 | 
   12 | 00m00s |   -0.00264 |      2.3119 | 
   13 | 00m00s |    0.00118 |      2.2212 | 
   14 | 00m00s |    0.00513 |      1.1176 | 
   15 | 00m00s |   -0.00449 |      2.3222 | 
   16 | 00m00s |    0.00445 |      1.0785 | 
   17 | 00m00s |    0.00015 |      2.1974 | 
   18 | 00m00s |    0.01138 |      1.5517 | 
   19 | 00m00s |   -0.01023 |      2.5616 | 
   20 | 00m00s |    0.01019 |      1.3063

In [53]:
fig = plot_gp(bo, x)

In [54]:
plt.savefig("/data_CMS/cms/wind/Plots/ggH_prior_optimization.pdf")